In [1]:
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import os
import json
from rsa import compute_rsm

In [2]:
# Loading dictionary of dtype to embed
with open('../../data/raw/dtype_to_embed.json', 'r') as f:
    dtype_to_embed = json.load(f)
    
brain_behav_names = dtype_to_embed['brain'] + dtype_to_embed['behavior']

# Iterating through pulled_embeds and finding union of all brain and behavior vocabs
embeds_path = '../../data/raw/embeds/'
brain_behav_union = set()
for f_name in tqdm(os.listdir(embeds_path)):
    
    embed_name = f_name.split('.')[0]
    if embed_name in brain_behav_names:
        vocab = set(pd.read_csv(embeds_path + f_name, index_col=0).index)
        brain_behav_union = brain_behav_union.union(vocab)

len(brain_behav_union)    

  0%|          | 0/26 [00:00<?, ?it/s]

45882

In [5]:
# Creating output directory
output_dir = '../../data/processed/rsms/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Computing RSMs
order_to_compute = dtype_to_embed['brain'] + dtype_to_embed['behavior'] + dtype_to_embed['text']
for embed_name in tqdm(order_to_compute):
    path = embeds_path + embed_name + '.csv'

    print(embed_name)
    embed = pd.read_csv(path, index_col=0)
    
    if embed.shape[0] != embed.shape[1]: # If not square, then it's a word embedding (not a RSM)
        embed = embed[embed.index.isin(brain_behav_union)]
        print(embed.shape)
        embed = compute_rsm(embed)

    embed.to_pickle(output_dir + f'{embed_name}.pkl')

  0%|          | 0/25 [00:00<?, ?it/s]

eye_tracking
(7486, 6)
EEG_speech
(1591, 130)
EEG_text
(3355, 104)
fMRI_speech_hyper_align
(579, 6)
fMRI_text_hyper_align
(1205, 1000)
microarray
(626, 15)
PPMI_SVD_SWOW
(11783, 300)
SGSoftMaxInput_SWOW
(11783, 300)
SGSoftMaxOutput_SWOW
(25442, 300)
ppmi_svd_south_florida


FileNotFoundError: [Errno 2] No such file or directory: '../../data/raw/embeds/ppmi_svd_south_florida.csv'